# Turbofan Engine Performance Model (Steady State)

by Noah Compton, Isabel Husted, and Jose Roque

Note: The main function explaination for evaluating an entire Turbofan engine can be found under "Full Turbofan Engine Performance."

- The function `my_engine` performs the performance calculation for every stage of the turbofan engine and will return the performance results for  each.

## Parts -or- Components
Parts are stored in independent files, and they all define a `class` with the Part name, for example, the file `compressor.py` will have the definition of the `class` Compressor.

For example, to import an specific Compressor, we export the Compressor `class` from the file `compressor.py`

In [16]:
exp = "Cmp20.PR = {'value': 20., 'units': '-' }"
print(exp)
exec(exp)

interested = [
                'name'     , 'inlet'    , 'outlet'   , 
                'Pt_in'    , 'Tt_in'    , 'W_in'     , 
                'Pt_out'   , 'Tt_out'   , 'W_out'    , 
                'PR'       , 'TR'       , 'N_mech'   ,
                'eff_poly' , 'eff_mech' , 'eff_isen' ,
                ]

print(interested)

'name' in interested

Cmp20.PR = {'value': 20., 'units': '-' }
['name', 'inlet', 'outlet', 'Pt_in', 'Tt_in', 'W_in', 'Pt_out', 'Tt_out', 'W_out', 'PR', 'TR', 'N_mech', 'eff_poly', 'eff_mech', 'eff_isen']


True

## Processes
Processes are `functions` that will be used throughout the Parts, make them iteract with each other and calculate certain paramenters. 

Processes are defined in an independent file `processes.py`

In [17]:
# Processes
from processes import *

For example, the function `check_units` in the processes file, will compare the units of two variables, this way when any class tries to add or substact variables it will check if they are compatible with each other to avoid calculation errors:

The following is an example of a unit mismatch and will return an error when the units are checked: 

In [19]:
var1 = {'value': 1.    , 'units': 'kg/s'} 
var2 = {'value': 2.    , 'units': 'kg'  } 
var3 = {'value': float , 'units':  str  }

check_units(var1, var2)

var3['value'] = var1['value'] + var2['value']
var3['units'] = var1['units']

ValueError: Units not consistent, var1 kg/s | var2 kg

Units for `var1` were kg/s and `var2` kg ... you can't add different dimensions!

Now, modifying `var2` to be kg/s ... 

In [20]:
var1 = {'value': 1.   , 'units': 'kg/s'} 
var2 = {'value': 2.   , 'units': 'kg/s'}
var3 = {'value': float, 'units':  str  }

check_units(var1, var2)

var3['value'] = var1['value'] + var2['value']
var3['units'] = var1['units']

var3

{'value': 3.0, 'units': 'kg/s'}

Since the units were correct, the addition was computed. Each class in this package has a built in `check_units` function for this reason. 

## Engine Definition

Engines are defined based on the template file: template_engine.py. Here, the parts are imported from their respective classes and named. 

Example of importing the parts for engine configuration:

In [21]:
from inlet       import Inlet
from fan         import Fan
from bypass      import Bypass
from compressor  import Compressor
from burner      import Burner
from turbine     import Turbine
from mixer       import Mixer
from nozzle      import Nozzle

This should be done at the top of the engine file, and only once per engine. Naming the parts is based on station number, but ultimately the choice is upt to the user. 

In [22]:
# Inlet
Int00 =      Inlet(name='Int10') # Inlet
    
# Core
Fan10 =        Fan(name='Fan10') # Fan (for Turbofan)
Cmp20 = Compressor(name='Cmp20') # Single Axial High Pressure Compressor
Brn30 =     Burner(name='Brn30') # Burner or Combustion Chamber
Trb40 =    Turbine(name='Trb40') # Single Axial High Pressure Turbine
    
# Bypass
Byp13 =     Bypass(name='Byp13') # Bypass right after Fan Station
Byp15 =     Bypass(name='Byp15') # Bypass downstream before Mixing
    
# Mixing Plane
Mix50 =      Mixer(name='Mix50') # Mixing Plane
    
# Nozzle / Exhaust
Noz70 =     Nozzle(name='Noz70') # Exhaust

After each component is defined, then known values are assigned, which can come from test data or design parameters

In [23]:
# Test Data / Design Parameters
# Inlet, Station: 00
Int00.u_in     = {'value': 600.    , 'units': 'm/s'  }
Int00.W_in     = {'value': 25.     , 'units': 'kg/s' }

# Compressor, Station: 20
Cmp20.PR       = {'value': 20.     , 'units': '-'    }
Cmp20.eff_poly = {'value': 0.9     , 'units': '-'    }

# Burner, Station: 30
Brn30.PR       = {'value': 0.95    , 'units': '-'    }
Brn30.eff_mech = {'value': 0.98    , 'units': '-'    }
Brn30.Tt_out   = {'value': 1673.   , 'units': 'K'    }

# Nozzle, Station: 70
Noz70.u_out    = {'value': 735.    , 'units': 'm/s'  }
Noz70.W_out    = {'value': 25.1070 , 'units': 'kg/s' }


Each component has a corresponding class, which is used to determine outlet conditions. 
- The input conditions for the inlet are the free stream conditions: stagnation pressure, stagnation temperature, and mach number. 
- Other input conditions are the inlet's characteristics, such as the incoming mass flow and the pressure ratio across the inlet.

First the Inlet class must be imported:

In [24]:
from inlet import *

Once the initial free stream conditions are set, the Inlet class will return the corresponding outlet conditions. The gas_dynamics package is used within the class to calculate properties such as the speed of sound and the total temperature and pressure.

The outputs are printed as a dictionary of a dictionary, with the corresponding units and values to each parameter.

In [25]:
Int10 = Inlet(name = 'Int10')

#Set initial free stream conditions
Int10.P_in   = {'value' : 12000  , 'units' : 'Pa'   }
Int10.T_in   = {'value' : 300    , 'units' : 'K'    }
Int10.XMN_in = {'value' : 2      , 'units' : '-'    }
#Set other inlet characteristics
Int10.W_in   = {'value' : 25     , 'units' : 'kg/s' }
Int10.PR     = {'value' : 0.9    , 'units' : '-'    }

Int10.calc()
import pprint
pprint.pprint(vars(Int10))

{'PR': {'units': '-', 'value': 0.9},
 'P_in': {'units': 'Pa', 'value': 12000},
 'Pt_in': {'units': 'Pa', 'value': 93893.38880240716},
 'Pt_out': {'units': 'Pa', 'value': 84504.04992216645},
 'TR': {'units': '-', 'value': 1.8},
 'T_in': {'units': 'K', 'value': 300},
 'Tt_in': {'units': 'K', 'value': 540.0},
 'Tt_out': {'units': 'K', 'value': 540.0},
 'W_in': {'units': 'kg/s', 'value': 25},
 'W_out': {'units': 'kg/s', 'value': 25},
 'XMN_in': {'units': '-', 'value': 2},
 'a_in': {'units': 'm/s', 'value': 347.1282183862326},
 'inlet': '',
 'name': 'Int10',
 'outlet': '',
 'u_in': {'units': 'm/s', 'value': 694.2564367724652}}


A similar procedure can be followed for the fan. However, the outlet conditions from the inlet are used as the inlet conditions for the fan.

In [26]:
from fan import *

Fan130 = Fan(name = 'fan130')

#Set fan inlet conditions using inlet outlet conditions
Fan130.Pt_in    = Int10.Pt_out
Fan130.Tt_in    = Int10.Tt_out
Fan130.W_in     = Int10.W_out

#Set fan characteristics
Fan130.PR       = {'value' : 1.5, 'units' : '-'}
Fan130.eff_poly = {'value' : 0.9, 'units' : '-'}

Fan130.calc()
pprint.pprint(vars(Fan130))

{'BPR': {'units': '-', 'value': 0.0},
 'PR': {'units': '-', 'value': 1.5},
 'Pt_in': {'units': 'Pa', 'value': 84504.04992216645},
 'Pt_out1': {'units': 'Pa', 'value': 126756.07488324968},
 'Pt_out2': {'units': 'Pa', 'value': 126756.07488324968},
 'TR': {'units': '', 'value': 1.137370571348326},
 'Tt_in': {'units': 'K', 'value': 540.0},
 'Tt_out1': {'units': 'K', 'value': 614.1801085280961},
 'Tt_out2': {'units': 'K', 'value': 614.1801085280961},
 'W_in': {'units': 'kg/s', 'value': 25},
 'W_out1': {'units': 'kg/s', 'value': 0.0},
 'W_out2': {'units': 'kg/s', 'value': 0.0},
 'eff_poly': {'units': '-', 'value': 0.9},
 'inlet': '',
 'name': 'fan130',
 'outlet': ''}


### LinkPorts:
Rather than manually inputting the outlet properties into the following class's input, a function can be used to simplify the process. The LinkPorts function converts the outlet temperature, pressure, and mass flow values to the inlet for the next station. The only alteration required to each of the classes is the addition of an "inlet" and "outlet" attribute, which are then defined by the LinkPorts function.

In [27]:
from processes import LinkPorts

#Use LinkPorts function to connect the inlet's outlet and fan's inlet
Int10.calc()
LinkPorts(Int10, Fan130)
Fan130.calc()

pprint.pprint(vars(Fan130))

{'BPR': {'units': '-', 'value': 0.0},
 'PR': {'units': '-', 'value': 1.5},
 'Pt_in': {'units': 'Pa', 'value': 84504.04992216645},
 'Pt_out1': {'units': 'Pa', 'value': 126756.07488324968},
 'Pt_out2': {'units': 'Pa', 'value': 126756.07488324968},
 'TR': {'units': '', 'value': 1.137370571348326},
 'Tt_in': {'units': 'K', 'value': 540.0},
 'Tt_out1': {'units': 'K', 'value': 614.1801085280961},
 'Tt_out2': {'units': 'K', 'value': 614.1801085280961},
 'W_in': {'units': 'kg/s', 'value': 25},
 'W_out1': {'units': 'kg/s', 'value': 0.0},
 'W_out2': {'units': 'kg/s', 'value': 0.0},
 'eff_poly': {'units': '-', 'value': 0.9},
 'inlet': 'Int10',
 'name': 'fan130',
 'outlet': ''}


### Inlets and Outlets:
LinkPorts takes an argument of (object1, object2): it sets object 1 as object 2's inlet and object 2 as object 1's outlet. When used consecutively with multiple objects, a class's inlet and outlet can be set as the previous and next station, respectively.

In [31]:
print(f'Fan130 inlet: {Fan130.inlet} outlet.')

Fan130 inlet: Int10 outlet.


## Details about components

#### a. Inlet

- The inlet brings free stream air into the engine

- The standard naming convention denotes free stream conditions with subscript 0, and output conditions with subscript 2.

    - i.e. T_0 is the free stream temperature and T_2 is the temperature at the outlet

#### b. Fan

- The fan splits the flow and directs air to the core and to the bypass, to then merge in the mixer

#### c. Bypass

- The bypass is where excess air not entering the core of the engine flows. This air flows around the core and joins with the core stream later in the 

    turbofan engine process.

#### d. Compressor

- Air flowing into the core of the engine first enters the compressor where pressure is exerted on the flow.

#### e. Burner

- Air is then subjected to fuel flow and combustion.

#### f. Turbine 

- Turbine takes flow in from the burner and outputs it to the mixer. 


#### g. Mixer

- Core flow and bypass flow are then mixed together before exiting the turbofan engine. 

#### h. Nozzle
- Nozzle takes flow from mixer and releases it downstream.

### Class Methods

Each station of a turbofan engine has a corresponding class in this package.

As mentioned before, the classes will be able to communicate with one another.

However, there are methods in the classes that require no input from other stations.

- This way, should the user need to calculate the conditions before and after just one station, they will be able to via these class methods.

### Examples:

The following examples show the class methods of the turbine station. 

- Note: The input conditions do not rely on the conditions of the other classes for these examples, only the turbine is being analyzed.

    - That being said, there are other methods that require the classes to communicate using `LinkPorts`.

First, we need to import the Turbine class and the gas_dynamics package.

In [32]:
from turbine import Turbine
import gas_dynamics as gd

The polytropic efficiency of the turbine can be calculated via a class method which uses temperature ratio and pressure ratio as inputs. 

- The format for this method is: `polytropic_efficiency(TR: float, PR: float, gas=gd.fluids.air)`

    - Note that the default gas being used is air. This can easily be changed by inputing a different fluid from the gas dynamics package.

In [33]:
efficiency = Turbine.polytropic_efficiency(TR=2.2, PR=20.7)
print(efficiency)

0.8715901379878442


If the user is instead searching for the temperature ratio, they can solve for it using pressure ratio and polytropic efficiency. 

- Format: `temp_ratio_from_poly_efficiency(PR: float, eff_poly: float, gas=gd.fluids.air)`

    - Again, the default gas is air.

In [34]:
temp_ratio = Turbine.temp_ratio_from_poly_efficiency(PR=20.7, eff_poly=0.91)
print(temp_ratio) 

2.1986307100568245


In the same way, the pressure ratio can also be calculated from temperature ratio and polytropic efficiency.

- Format: `pressure_ratio_from_poly_efficiency(TR: float, eff_poly: float, gas=gd.fluids.air)`

In [35]:
pressure_ratio = Turbine.pressure_ratio_from_poly_efficiency(TR=2.4, eff_poly=0.94)
print(pressure_ratio)

24.339296054725345


Total conditions are a crucial part of engine analysis. Therefore, there are methods that can calculate them.

Given an initial total pressure, the temperature ratio across the turbine, and the efficiency of the turbine, the following class method can evaluate the resulting total pressure exiting the turbine.

- Format: `Pt_out_from_poly_efficiency(Pt_in: float, TR:float, eff_poly:float, gas=gd.fluids.air)`

In [36]:
Pt5 = Turbine.Pt_out_from_poly_efficiency(Pt_in=2000, TR=2.2, eff_poly=efficiency)
print(Pt5)

41399.99999999999


Similarly, total temperature can be evaluated.

- Format: `Tt_out_from_poly_efficiency(Tt_in: float, PR: float, eff_poly: float, gas=gd.fluids.air)`

In [37]:
Tt5 = Turbine.Tt_out_from_poly_efficiency(Tt_in=450, PR=20.7, eff_poly=efficiency)
print(Tt5)

539.0


Although total conditions are important, sometimes static conditions are needed, especially in the turbine. 

Therefore, there is a method of the turbine class which calculates the static pressure at the exit.

- Format: `static_pressure_out(Pt_out: float, Tt_out: float, mach_out: float, gas=gd.fluids.air)`

In [38]:
P5 = Turbine.static_pressure_out(Pt_out=Pt5, Tt_out=Tt5, mach_out=0.5)
print(P5)

34900.99386249369


The last example shows the calculation of the speed of sound at the exit of the turbine. Since it is dependent on 
temperature, calculating the sonic velocity within the engine can provide information on how well the turbofan is performing.
- Format: `speed_of_sound_out(Tt_out: float, mach_out: float, gas=gd.fluid.air)`
    - Note: This calculation requires the use of the gas gamma value, $\gamma$, as well as the gas constant, R.
        - Previous methods also require $\gamma$. This is why the integration of the gas_dynamics package is essential, as it  defines these constants, as well as other parameters, for any gas in this package.

In [39]:
speed_o_sound = Turbine.speed_of_sound_out(Tt_out=500, mach_out=0.5)
print(speed_o_sound)

437.34044709661447


Example of using a gas other than the default air:

In [40]:
speed_o_sound_argon = Turbine.speed_of_sound_out(Tt_out=500, mach_out=0.5, gas=gd.fluids.argon)
print(speed_o_sound_argon)

400.32282244258414


Note the difference between this result and the one above, which used the defualt gas air as an input.

All methods have the option to input a different gas thanks to the gas_dynamics package, although in reality the gas being used will almost always be air.

## Full Turbofan Engine Performance

To use the package to assess an engine, first import the engine file. This example uses a custom engine, template_engine.py, in which all components and design parameters are defined.

The `my_engine()` function will run the entire program and relevant test data can be extracted.

In [41]:
import template_engine as engine

Int00, Fan10, Cmp20, Brn30, Trb40, Mix50, Noz70, Fn, TSFC = engine.my_engine()

Any stage within the turbofan can be analyzed by accessing the respective dictionary.

In [42]:
Int00.Pt_out

{'value': 84504.04992216645, 'units': 'Pa'}

## Performance Parameters:
processes contain a few functions to calculate performance paramenters of the system, such as:
1) Gross Thrust -> `gross_thrust(nozzle)`
2) Ram Drag -> `ram_drag(inlet)`
3) Net Thrust -> `net_thrust(inlet, nozzle)`
4) TSFC (Thrust-specific-fuel-conpsumption) -> `tsfc(inlet, burner, nozzle)`
5) Thermal, Propulsive and Overall System Efficiency -> `efficiency(inlet, burner, nozzle)`

All functions will take as input the specific class

### Net Thrust
Net Thrust can be calculated using net_thrust(inlet: Inlet, nozzle: Nozzle):
- Format `net_thrust(inlet, nozzle)`

The function will take as input the `inlet` and `nozzle` of the Engine and calculate the net thrust for the given condition

First import the processes file, which includes the aforementioned functions.

In [43]:
from processes import *

Now we can calculate the net thrust, which uses gross thrust and ram drag calculations within its function.

In [44]:
Fn    = net_thrust(Int00, Noz70)
Fn

{'value': 4502.889794108069, 'units': 'N'}